In [ ]:
//| export

import { assert } from "asserts";

# Utilities

Useful helpers

## Get project root

Figure out where project root is. Useful for normalizing paths to things like DB
file

In [ ]:
//| export

import path from "node:path";

export const getProjectRoot = (
  dir: string = Deno.cwd(),
  d = 0,
  maxD = 10
): string => {
  if (d >= maxD) throw new Error("max depth reached");

  try {
    const f = path.join(dir, "deno.json");
    Deno.lstatSync(f);
    return path.dirname(f);
  } catch {
    return getProjectRoot(path.join(dir, "../"), d + 1);
  }
};

In [ ]:
path.join(getProjectRoot(), "data/tc.db");

"/Users/philip/projects/tinychat/data/tc.db"

## Unslopify imports and exports

atproto codegens generate modules with sloppy imports like this:

```ts
import * as ChatTinychatActorProfile from "./types/chat/tinychat/actor/profile";
import * as ChatTinychatServer from "./types/chat/tinychat/server";
```

We need to convert this to something like this

```ts
import * as ChatTinychatActorProfile from "./types/chat/tinychat/actor/profile.ts";
import * as ChatTinychatServer from "./types/chat/tinychat/server.ts";
```

Let's create `processLine` to process one a line from ts module

In [ ]:
//| export

const processLine = (line: string): string => {
  if (!line.trim().match(/^import|export/ig)) {
    return line;
  }
  const module = line.split("from").pop()?.trim().replaceAll(/'|"|;/ig, "");
  if (!module || !module.startsWith(".") || module.endsWith(".ts")) {
    return line;
  }
  return line.replace(module!, `${module}.ts`);
};

In [ ]:
import { assertEquals } from "asserts";

Deno.test("processLine", () => {
  assertEquals(
    processLine(
      `export * as ComAtprotoTempRequestPhoneVerification from "./types/com/atproto/temp/requestPhoneVerification";`,
    ),
    `export * as ComAtprotoTempRequestPhoneVerification from "./types/com/atproto/temp/requestPhoneVerification.ts";`,
  );
  assertEquals(
    processLine(
      `import * as ComAtprotoTempRequestPhoneVerification from "./types/com/atproto/temp/requestPhoneVerification";`,
    ),
    `import * as ComAtprotoTempRequestPhoneVerification from "./types/com/atproto/temp/requestPhoneVerification.ts";`,
  );
  assertEquals(processLine(`export class ChatNS {`), `export class ChatNS {`);
  assertEquals(
    processLine(
      `import * as ComAtprotoTempRequestPhoneVerification from "./types/com/atproto/temp/requestPhoneVerification.ts";`,
    ),
    `import * as ComAtprotoTempRequestPhoneVerification from "./types/com/atproto/temp/requestPhoneVerification.ts";`,
  );
});

`processFile` runs conversion of the whole file

In [ ]:
//| export

const processFile = async (file: string): Promise<string> => {
  const text = await Deno.readTextFile(file);
  const modifiedText = text.split("\n").map(processLine).join("\n");
  await Deno.writeTextFile(file, modifiedText);
  return modifiedText;
};


In [ ]:
Deno.test("processFile", async () => {
  const td = await Deno.makeTempDir({});
  await Deno.writeTextFile(
    `${td}/test.ts`,
    `
    export * as Foo from "./foo";
    import { bar } from "./bar";

    export class ChatNS {
      public foo: Foo;
      public bar: bar;
    }
  `,
  );
  await processFile(`${td}/test.ts`);
  assertEquals(
    await Deno.readTextFile(`${td}/test.ts`),
    `
    export * as Foo from "./foo.ts";
    import { bar } from "./bar.ts";

    export class ChatNS {
      public foo: Foo;
      public bar: bar;
    }
  `,
  );
});

`unslopifyModules` is the main function to recursively process modules in a
directory

In [ ]:
//| export

import { walk } from "jsr:@std/fs/walk";

export const unslopifyModules = async (dir: string) => {
  for await (const dirEntry of walk(dir, { exts: ["ts"] })) {
    await processFile(dirEntry.path);
  }
};

## Sleep

Async sleep helper

In [ ]:
//| export

export const sleep = (ms: number) =>
  new Promise((resolve) => setTimeout(resolve, ms));

# Assert with a timeout

Run assert after waiting for delay ms

In [ ]:
//| export

export async function assertWithWait<T>(
  fn: () => Promise<T>,
  expected: T,
  delay: number = 5000,
) {
  await sleep(delay);
  assert((await fn()) === expected);
}

In [ ]:
Deno.test("assertWithWait", async () => {
  await assertWithWait(() => Promise.resolve(2), 2, 5000);
});

# Short ID from at-uri

In [ ]:
//| export

export const shortIdFromAtUri = (atUri: string) => {
  return atUri.split("/").pop();
}

In [ ]:
import {assertEquals} from "asserts";

Deno.test("shortIdFromAtUri", () => {
  assertEquals(shortIdFromAtUri("at://did:plc:ubdeopbbkbgedccgbum7dhsh/chat.tinychat.server/3lfu4indvy72b"), "3lfu4indvy72b");
});

# URL helpers for ServerView

In [ ]:
//| export

import { ids } from "tinychat/api/lexicons.ts";
import { ChannelView } from "tinychat/api/types/chat/tinychat/server/defs.ts";

export const serverAtURIFromUrl = (url: string) => {
  const parts = url.split("?")[0].split("/chat")[1].replace(/^\//ig, "").split("/");
  return `at://did:plc:${parts[0]}/${ids.ChatTinychatCoreServer}/${parts[1]}`;
}

export const urlFromServerAtURI = (atUri: string) => {
  const parts = atUri.split(ids.ChatTinychatCoreServer);
  //@ts-ignore yolo
  const did = parts[0].split(":").pop().replace("/", "");
  //@ts-ignore yolo
  const rkey = parts[1].replace("/", "");
  return `/chat/${did}/${rkey}`;  
}


export const urlForChannelMessageList = (channel: ChannelView) => {
  return urlFromServerAtURI(channel.server).replace("/chat", "/messages/list") + "/" + channel.id;
}

export const parseURLForChannelMessageList = (url: string): { server: string, channel: string }  => {
  const parts = url.replace("/messages/list/", "").split("/"); 
  // "/messages/list/ubdeopbbkbgedccgbum7dhsh/3lgawfvbbtx2b/abc";
  console.log(parts);
  return {
    server: `at://did:plc:${parts[0]}/${ids.ChatTinychatCoreServer}/${parts[1]}`,
    channel: parts[2]
  };
}


In [ ]:
import { assertEquals } from "asserts";

Deno.test("serverAtURIFromUrl", () => {
  assertEquals(
    serverAtURIFromUrl("https://tinychat.ngrok.app/chat/ubdeopbbkbgedccgbum7dhsh/3lgawfvbbtx2b"),
    "at://did:plc:ubdeopbbkbgedccgbum7dhsh/chat.tinychat.core.server/3lgawfvbbtx2b"
  );
  assertEquals(
    serverAtURIFromUrl(
      "https://tinychat.ngrok.app/chat/ubdeopbbkbgedccgbum7dhsh/3lgawfvbbtx2b?a=1&b=2"
    ),
    "at://did:plc:ubdeopbbkbgedccgbum7dhsh/chat.tinychat.core.server/3lgawfvbbtx2b"
  );
});

Deno.test("urlFromServerAtURI", () => {
  assertEquals(
    urlFromServerAtURI("at://did:plc:ubdeopbbkbgedccgbum7dhsh/chat.tinychat.core.server/3lgawfvbbtx2b"),
    "/chat/ubdeopbbkbgedccgbum7dhsh/3lgawfvbbtx2b"
  );
});

Deno.test("urlForChannelMessageList", () => {
  assertEquals(
    urlForChannelMessageList({
      id: "abc",
      server:
        "at://did:plc:ubdeopbbkbgedccgbum7dhsh/chat.tinychat.core.server/3lgawfvbbtx2b",
      name: "foo",
    }),
    "/messages/list/ubdeopbbkbgedccgbum7dhsh/3lgawfvbbtx2b/abc"
  );
});

Deno.test("parseURLForChannelMessageList", () => {
  assertEquals(
    parseURLForChannelMessageList("/messages/list/ubdeopbbkbgedccgbum7dhsh/3lgawfvbbtx2b/abc"),
    { server:"at://did:plc:ubdeopbbkbgedccgbum7dhsh/chat.tinychat.core.server/3lgawfvbbtx2b", channel: "abc"}
  );
});